In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import csv
import math
import random

from BSE import market_session

def plot_trades(trial_id):
    prices_fname = trial_id + '_tape.csv'
    x = np.empty(0)
    y = np.empty(0)
    with open(prices_fname, newline='') as csvfile:
        reader = csv.reader(csvfile)
        for row in reader:
            time = float(row[1])
            price = float(row[2])
            x = np.append(x,time)
            y = np.append(y,price)

    plt.plot(x, y, 'x', color='black') 
    
# Use this to run an experiment n times and plot all trades
def n_runs_plot_trades(n, trial_id, start_time, end_time, traders_spec, order_sched):
    x = np.empty(0)
    y = np.empty(0)

    for i in range(n):
        trialId = trial_id + '_' + str(i)
        tdump = open(trialId + '_avg_balance.csv','w')

        market_session(trialId, start_time, end_time, traders_spec, order_sched, tdump, True, False)
        
        tdump.close()

        with open(trialId + '_tape.csv', newline='') as csvfile:
            reader = csv.reader(csvfile)
            for row in reader:
                time = float(row[1])
                price = float(row[2])
                x = np.append(x,time)
                y = np.append(y,price)

    plt.plot(x, y, 'x', color='black');

# !!! Don't use on it's own   
def getorderprice(i, sched, n, mode):
    pmin = min(sched[0][0], sched[0][1])
    pmax = max(sched[0][0], sched[0][1])
    prange = pmax - pmin
    stepsize = prange / (n - 1)
    halfstep = round(stepsize / 2.0)

    if mode == 'fixed':
        orderprice = pmin + int(i * stepsize)
    elif mode == 'jittered':
        orderprice = pmin + int(i * stepsize) + random.randint(-halfstep, halfstep)
    elif mode == 'random':
        if len(sched) > 1:
            # more than one schedule: choose one equiprobably
            s = random.randint(0, len(sched) - 1)
            pmin = min(sched[s][0], sched[s][1])
            pmax = max(sched[s][0], sched[s][1])
        orderprice = random.randint(pmin, pmax)
    return orderprice    

# !!! Don't use on it's own
def make_supply_demand_plot(bids, asks):
    # total volume up to current order
    volS = 0
    volB = 0

    fig, ax = plt.subplots()
    plt.ylabel('Price')
    plt.xlabel('Quantity')
    
    pr = 0
    for b in bids:
        if pr != 0:
            # vertical line
            ax.plot([volB,volB], [pr,b], 'r-')
        # horizontal lines
        line, = ax.plot([volB,volB+1], [b,b], 'r-')
        volB += 1
        pr = b
    if bids:
        line.set_label('Demand')
        
    pr = 0
    for s in asks:
        if pr != 0:
            # vertical line
            ax.plot([volS,volS], [pr,s], 'b-')
        # horizontal lines
        line, = ax.plot([volS,volS+1], [s,s], 'b-')
        volS += 1
        pr = s
    if asks:
        line.set_label('Supply')
        
    if bids or asks:
        plt.legend()
    plt.show()

# Use this to plot supply and demand curves from supply and demand ranges and stepmode
def plot_sup_dem(seller_num, sup_ranges, buyer_num, dem_ranges, stepmode):
    asks = []
    for s in range(seller_num):
        asks.append(getorderprice(s, sup_ranges, seller_num, stepmode))
    asks.sort()
    bids = []
    for b in range(buyer_num):
        bids.append(getorderprice(b, dem_ranges, buyer_num, stepmode))
    bids.sort()
    bids.reverse()
    
    make_supply_demand_plot(bids, asks) 

# plot sorted trades, useful is some situations - won't be used in this worksheet
def in_order_plot(trial_id):
    prices_fname = trial_id + '_tape.csv'
    y = np.empty(0)
    with open(prices_fname, newline='') as csvfile:
        reader = csv.reader(csvfile)
        for row in reader:
            price = float(row[2])
            y = np.append(y,price)
    y = np.sort(y)
    x = list(range(len(y)))

    plt.plot(x, y, 'x', color='black')   

# plot offset function
def plot_offset_fn(offset_fn, total_time_seconds):   
    x = list(range(total_time_seconds))
    offsets = []
    for i in range(total_time_seconds):
        offsets.append(offset_fn(i))
    plt.plot(x, offsets, 'x', color='black') 

In [ ]:
tdump=open('new.csv','w')
trial_id = 'pula_mea2'
dump_all = True
verbose = False

In [ ]:
sup_range = (60, 60)
dem_range = (140, 140)
start_time = 0 
end_time = 60 * 60 * 24 * 5

supply_schedule = [{'from': start_time, 'to': end_time, 'ranges': [sup_range], 'stepmode': 'fixed'}]
demand_schedule = [{'from': start_time, 'to': end_time, 'ranges': [dem_range], 'stepmode': 'fixed'}]
#plot_sup_dem(10, [sup_range], 10, [dem_range], 'fixed')
order_sched = {'sup': supply_schedule, 'dem': demand_schedule, 'interval': 5, 'timemode': 'drip-poisson' }

list_averages_k4 = {}
list_averages2_k4 = {}
F_values = [1.1, 1.2, 1.3, 1.4, 1.5, 1.6, 1.7, 1.8, 1.9, 2.0]

for k in [4]:
    for F in F_values:
        trial = 0
        list_exp = np.empty(0)
        list_prde = np.empty(0)

        while (trial < 3):
            trial_id = f'exp_k={k}_F={F}_trail={trial}'
            buyers_spec = [('PRDE_EXP', 10, {'k': k, 's_min': -1.0, 's_max': +1.0, 'F': F}),
                ('PRDE', 10, {'k': 4,'s_min': -1.0, 's_max': +1.0, 'F': 0.8})]
                
            sellers_spec = buyers_spec
            traders_spec = {'sellers':sellers_spec, 'buyers':buyers_spec}
            
            market_session(trial_id, start_time, end_time, traders_spec, order_sched, tdump, dump_all, verbose)
            
            sum_prde_buy = 0
            sum_exp_buy = 0
            sum_prde_sell = 0
            sum_exp_sell = 0
            nr_trader = 0
            y_prde = np.empty(0)
            y_exp = np.empty(0)
            
            with open(trial_id + '_exp.csv', newline='') as csvfile:
                reader = csv.reader(csvfile)
                for row in reader:
                    time = float(row[0])
                    sum_prde_buy += float(row[1])
                    sum_prde_sell += float(row[2])
                    sum_exp_buy += float(row[3])
                    sum_exp_sell += float(row[4]) 
                    nr_trader += 1
                    y_prde = np.append(y_prde, (sum_prde_buy+ sum_prde_sell)/nr_trader)
                    y_exp = np.append(y_exp, (sum_exp_buy + sum_exp_sell)/nr_trader)

            list_exp = np.append(list_exp, y_exp[-1])
            list_prde = np.append(list_prde, y_prde[-1])

            trial += 1

        average_exp = np.mean(list_exp)
        average_prde = np.mean(list_prde)
        list_averages_k4[F] = average_exp 
        list_averages2_k4[F] = average_prde


In [ ]:
print('PRDE custom:',list_averages_k4)

In [ ]:
print('PRDE k=4, F=0.8', list_averages2_k4)

In [ ]:
# Multiple trials with best configuration.
sup_range = (60, 60)
dem_range = (140, 140)
start_time = 0 
end_time = 60 * 60 * 24 * 5

supply_schedule = [{'from': start_time, 'to': end_time, 'ranges': [sup_range], 'stepmode': 'fixed'}]
demand_schedule = [{'from': start_time, 'to': end_time, 'ranges': [dem_range], 'stepmode': 'fixed'}]
#plot_sup_dem(10, [sup_range], 10, [dem_range], 'fixed')
order_sched = {'sup': supply_schedule, 'dem': demand_schedule, 'interval': 5, 'timemode': 'drip-poisson' }

trial = 0
k = 4
F = 1.7
list_exp = np.empty(0)
list_prde = np.empty(0)

while (trial < 30):
    trial_id = f'best2_exp_k={k}_F={F}_trail={trial}'
    buyers_spec = [('PRDE_EXP', 10, {'k': k, 's_min': -1.0, 's_max': +1.0, 'F': F}),
        ('PRDE', 10, {'k': 4,'s_min': -1.0, 's_max': +1.0, 'F': 0.8})]            
    sellers_spec = buyers_spec
    traders_spec = {'sellers':sellers_spec, 'buyers':buyers_spec}
            
    market_session(trial_id, start_time, end_time, traders_spec, order_sched, tdump, dump_all, verbose)
            
    sum_prde_buy = 0
    sum_exp_buy = 0
    sum_prde_sell = 0
    sum_exp_sell = 0
    nr_trader = 0
    y_prde = np.empty(0)
    y_exp = np.empty(0)
            
    with open(trial_id + '_exp.csv', newline='') as csvfile:
        reader = csv.reader(csvfile)
        for row in reader:
            time = float(row[0])
            sum_prde_buy += float(row[1])
            sum_prde_sell += float(row[2])
            sum_exp_buy += float(row[3])
            sum_exp_sell += float(row[4]) 
            nr_trader += 1
            y_prde = np.append(y_prde, (sum_prde_buy+ sum_prde_sell)/nr_trader)
            y_exp = np.append(y_exp, (sum_exp_buy + sum_exp_sell)/nr_trader)

    list_exp = np.append(list_exp, y_exp[-1])
    list_prde = np.append(list_prde, y_prde[-1])

    trial += 1


In [ ]:
list_exp

In [ ]:
list_prde